In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import utils
import duckdb

DATA_DIR = 'data/imdb/'

schema = utils.read_json(os.path.join(DATA_DIR, 'schema.json'))

con = duckdb.connect('dbs/imdb.duckdb', read_only=True)

def check(table_size, type, min_val=None, max_val=None, is_nullable=None, null_count=None, distinct_count=None, values=None):
  assert null_count is not None
  assert distinct_count is not None

  # NOTE: `is_nullable` comes from the schema!
  if null_count == table_size:
    return {
      'min' : None,
      'max' : None,
      '#distinct' : 1,
      'is-nullable' : is_nullable,
      'has-null' : True,
      'only-null' : True
    }
  if type == 'integer':
    return {
      'min' : int(min_val),
      'max' : int(max_val),
      '#distinct' : int(distinct_count),
      'is-nullable' : is_nullable,
      'has-null' : (null_count != 0),
      'only-null' : False
    }
  elif type == 'varchar' or type == 'char':
    return {
      'min' : None,
      'max' : None,
      '#distinct' : int(distinct_count),
      'is-nullable' : is_nullable,
      'has-null' : (null_count != 0),
      'only-null' : False,
      'values' : values
    }
  else:
    assert 0
    return {}

def analyze_col(tn, table_size, col):
  print(f'[analyze] {tn} {col}')

  type = schema[tn]['columns'][col]['type']
  is_nullable = not schema[tn]['columns'][col]['not-null']

  # NOTE: `COUNT(DISTINCT column)` does *not* consider NULLs!
  # NOTE: However, DISTINCT alone *does* consider NULLs.
  if type == 'integer':
    ret = con.sql(f"""
      select
        MIN({col}) as min_,
        MAX({col}) max_,
        SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) as null_count,
        COUNT(DISTINCT {col}) as distinct_count
      from {tn};
    """).df()
    min_val = ret['min_'][0]
    max_val = ret['max_'][0]

    # This is later used to test if we have just NULLs.
    null_count = int(ret['null_count'][0])

    # NOTE: We also include NULLs here.
    # NOTE: However, we decide this only based on the nullable-option, since the current data might not have any NULLs.
    distinct_count = ret['distinct_count'][0] + int(is_nullable)
    return check(table_size, type, min_val=min_val, max_val=max_val, is_nullable=is_nullable, null_count=null_count, distinct_count=distinct_count)
  elif type == 'varchar' or type == 'char':
    ret = con.sql(f"""
      select
        SUM(CASE WHEN {col} IS NULL THEN 1 ELSE 0 END) as null_count,
        COUNT(DISTINCT {col}) as distinct_count
      from {tn};
    """).df()
    
    # This is later used to test if we have just NULLs.
    null_count = int(ret['null_count'][0])

    # NOTE: We also include NULLs here.
    # NOTE: However, we decide this only based on the nullable-option, since the current data might not have any NULLs.
    distinct_count = ret['distinct_count'][0] + int(is_nullable)

    print(distinct_count)

    # Collect the unique values.
    values = None
    if distinct_count + int(is_nullable) <= 256:
      values = con.sql(f"""
        SELECT DISTINCT {col} as set
        FROM {tn};
      """).df()
      values = list(values['set'].values)
    return check(table_size, type, is_nullable=is_nullable, null_count=null_count, distinct_count=distinct_count, values=values)
  else:
    return {}

analysis = {}
for tn in schema:
  table_size = utils.get_table_size(con, tn)
  analysis[tn] = { '#size' : int(table_size) }
  for col in schema[tn]['columns']:
    analysis[tn][col] = analyze_col(tn, table_size, col)

# Close the connection.
con.close()

# utils.print_dict(analysis)

utils.write_json(os.path.join(DATA_DIR, 'data-analysis.json'), analysis)